In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
#warnings.simplefilter("ignore", category=DeprecationWarning)
#warnings.simplefilter("ignore", category=PendingDeprecationWarning)
#warnings.simplefilter("ignore", category=ImportWarning)
warnings.filterwarnings("ignore")


# Standard library imports.
from datetime import timedelta, datetime
import pprint
import pickle
import json

# Related third party imports.
import matplotlib.pyplot as plt
import numpy as np

# Local application/library specific imports.
from cerebralcortex.cerebralcortex import CerebralCortex
import util

In [2]:
# Load all user IDs
with open('users.json', 'r') as f:
    USR_IDS = json.load(f)


# Load CerebralCortex Configurations
CC = CerebralCortex("/home/mperf/sandeep/CerebralCortex-DockerCompose/cc_config_file/cc_vagrant_configuration.yml")

pp = pprint.PrettyPrinter(indent=2)

# Extract working beacon data 

In [3]:
# Dictionary: usr_id -> list of datapoints
id_to_work_bcn = dict()

for usr_id in USR_IDS:

    id_to_work_bcn[usr_id] = []
    usr_streams = CC.get_user_streams(usr_id)
    stream_labels = list(usr_streams.keys())

    # Get beacon stream at work place
    work_bcn_stream_labels = util.extract_matched_labels(stream_labels, ['BEACON', 'WORK'])
    if work_bcn_stream_labels:
        work_bcn_stream = usr_streams[work_bcn_stream_labels[0]]
        
        # Enumerate stream id in each ble beacon stream
        for stream_id in work_bcn_stream['stream_ids']:
            stream_days = CC.get_stream_days(stream_id)
            
            
            # Get all work beacon data from a user in chronological order 
            for i, stream_day in enumerate(stream_days):
                ds = CC.get_stream(stream_id, usr_id, stream_day)
                id_to_work_bcn[usr_id].extend(ds.data)
    id_to_work_bcn[usr_id].sort()


/home/mperf/sandeep/CerebralCortex/cerebralcortex/cerebralcortex.py:189: DeprecationWarning: PLEASE USE stream_ids KEY IN DICT OBJECT TO GET ALL STREAM IDS OF A STREAM NAME. Identifier key will be removed in CerebralCortex version 2.2.4.
  warnings.warn("PLEASE USE stream_ids KEY IN DICT OBJECT TO GET ALL STREAM IDS OF A STREAM NAME. Identifier key will be removed in CerebralCortex version 2.2.4.", DeprecationWarning)
/home/mperf/sandeep/CerebralCortex/cerebralcortex/cerebralcortex.py:189: DeprecationWarning: PLEASE USE stream_ids KEY IN DICT OBJECT TO GET ALL STREAM IDS OF A STREAM NAME. Identifier key will be removed in CerebralCortex version 2.2.4.
  warnings.warn("PLEASE USE stream_ids KEY IN DICT OBJECT TO GET ALL STREAM IDS OF A STREAM NAME. Identifier key will be removed in CerebralCortex version 2.2.4.", DeprecationWarning)
/home/mperf/sandeep/CerebralCortex/cerebralcortex/cerebralcortex.py:189: DeprecationWarning: PLEASE USE stream_ids KEY IN DICT OBJECT TO GET ALL STREAM IDS 

# Extract groundtrtuth from working beacon data

In [4]:
# key: usr_id, value: list of (start time: datetime, end time: datetime)
at_desk = dict()

# time difference threshold value in second for two consecutive beacons to be considered as a stream.
tdf_th = 25.0

# distance threshold value in meter for a beacon to be considered as valid.
dist_th = 2.0 

for usr_id in USR_IDS:
    work_bcns = id_to_work_bcn[usr_id]
    at_desk[usr_id] = list()
    
    start_bcn = None
    prev_bcn = None

    for bcn in work_bcns:
        # Filter out beacon with ditance
        dist = bcn.sample[0]
        if dist > dist_th:
            continue
        if start_bcn is None:
            start_bcn = bcn
        if prev_bcn is None:
            prev_bcn = bcn

        # Time gap between current bcn and prev bcn
        tdf = bcn.start_time - prev_bcn.start_time
        
        
        if tdf.total_seconds() > tdf_th:
            # Current beacon is too long time away from previous beacon
            # Save the stream
            at_desk[usr_id].append((start_bcn, prev_bcn))
            
            # Start a new stream
            start_bcn = bcn
        prev_bcn = bcn
     

# Save processed raw data and groundtruth

In [ ]:
with open('work_bcn.pkl', 'wb') as f:
    pickle.dump(id_to_work_bcn, f)
with open('at_desk_groundtruth.pkl', 'wb') as f:
    pickle.dump(at_desk, f)

In [23]:
for i, usr_id in enumerate(at_desk.keys()):
    print('------------usr %d----------' % i)
    for bcn_pair in at_desk[usr_id]:
        tdf = bcn_pair[1].start_time - bcn_pair[0].start_time
        print(tdf.total_seconds())

------------usr 0-------
------------usr 1-------
0.0
40.013
0.0
0.0
0.0
0.0
59.984
40.013
0.0
0.0
19.996
0.0
0.0
40.005
0.0
0.0
0.0
7.121
0.0
0.0
0.0
0.0
19.978
19.985
0.0
0.0
0.0
19.967
40.024
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.152
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.006
0.0
0.0
40.027
0.0
0.0
0.0
20.015
0.0
0.0
0.0
0.0
0.0
37.872
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.01
39.986
0.0
0.0
39.987
0.0
99.979
0.0
59.984
312.858
819.956
0.0
260.006
0.0
80.003
135.38
140.017
80.025
280.007
0.0
100.001
80.066
19.995
20.001
160.013
19.979
539.921
39.958
6.537
199.98
220.019
129.202
198.9
359.995
174.995
280.018
78.991
0.0
270.255
0.0
5.111
33.965
633.722
537.421
1440.133
0.0
19.992
0.0
19.997
60.006
13.865
80.016
20.019
20.011
0.0
0.0
20.005
39.996
100.008
18.983
20.006
2459.209
840.483
238.81
79.985
4319.253
2440.877
4654.378
92.105
39.992
1939.32
1520.79
0.0
2821.374
25.39
0.0
2.898
3.515
0.0
26.907
0.0
10.184
8.208
2.038
14

99.269
79.988
0.0
120.021
40.008
0.0
19.962
0.0
19.99
79.993
0.0
0.0
0.0
0.0
99.978
0.0
0.0
40.024
0.0
139.998
39.977
40.018
0.0
59.991
100.019
160.019
19.997
0.0
20.021
0.0
20.036
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.009
59.974
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
79.975
40.051
20.018
60.04
0.0
0.0
20.02
39.996
0.0
59.971
79.97
19.986
318.668
80.013
20.001
0.0
80.025
39.983
100.015
19.986
0.0
0.0
140.017
19.96
19.976
0.0
39.999
20.015
40.041
39.999
0.0
19.987
0.0
0.0
20.009
0.0
0.0
140.011
19.963
59.998
39.971
79.997
20.002
59.991
60.03
19.984
0.0
0.0
60.004
0.0
0.0
20.018
0.0
0.0
40.008
0.0
0.0
0.0
40.042
0.0
0.0
58.964
0.0
59.986
79.989
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.046
100.021
20.001
79.995
60.003
100.03
39.984
378.543
0.0
219.988
180.011
200.514
0.0
0.0
40.05
0.0
20.0
0.0
0.0
79.968
40.006
19.989
59.99
40.024
99.962
60.065
0.0
0.0
19.997
0.0
19.996
40.024
0.0
99.984
59.988
0.0
80.024
0.0
0.0
0.0
59.993
19.957
40.037
20.038
0.0
0.0
19.969
0.0
0.0
0.0
0.0
40.038

0.0
0.0
39.961
0.0
20.009
0.0
0.0
0.0
0.0
19.987
79.988
0.0
20.372
18.982
0.0
39.988
60.022
0.0
0.0
0.0
0.0
20.389
0.0
0.0
18.969
0.0
559.985
0.0
0.0
79.999
0.0
0.0
0.0
0.0
20.009
0.0
0.0
0.0
0.0
19.987
59.976
20.052
19.991
0.0
540.004
20.013
0.0
160.026
259.029
19.968
60.015
0.0
39.998
40.0
0.0
120.636
0.0
20.018
40.001
0.0
0.0
18.962
0.0
0.0
19.964
39.975
47.085
0.0
0.0
0.0
20.002
0.0
60.008
0.0
0.0
20.008
5.068
80.038
159.994
359.993
40.0
39.976
60.004
200.005
0.0
20.007
39.297
79.991
259.988
20.0
28.541
99.974
0.0
0.0
0.0
0.0
0.0
20.008
300.0
19.982
19.988
200.778
0.0
119.997
60.006
------------usr 2-------
36.687
591.626
1802.522
134.816
82.554
15.039
79.847
427.115
54.066
162.2
47.742
258.56
841.032
3803.666
10.526
416.364
148.533
4001.896
515.584
4610.193
399.551
1790.989
718.762
12812.872
1969.324
5946.838
8291.585
1763.866
1797.843
2102.62
5914.746
9.457
0.003
31.692
286.233
10408.826
29.668
620.137
586.413
1280.693
12675.177
3952.971
44.79
27.189
48.495
125.641
2.165
867.366


39.976
0.0
60.005
0.0
0.0
0.0
39.988
0.0
79.985
60.023
0.0
0.0
0.0
0.0
99.984
19.998
199.99
0.0
0.0
0.0
20.005
0.0
40.011
0.0
59.998
119.965
257.944
539.952
20.006
0.0
18.949
0.0
39.965
0.0
0.0
0.0
0.0
479.981
0.0
0.0
0.0
0.0
0.0
0.0
99.711
0.0
0.0
0.0
0.0
0.0
0.0
20.036
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.002
19.984
0.0
120.015
100.019
20.002
299.996
320.008
49.739
0.0
10.461
53.809
3.435
22.472
150.487
0.0
182.58
0.0
19.993
0.0
20.045
100.419
340.01
140.014
0.0
20.0
0.0
19.999
0.0
0.0
120.006
60.048
0.0
39.991
19.958
0.0
20.027
0.0
659.975
220.006
259.988
40.0
0.0
220.013
0.0
0.0
0.0
280.946
0.0
22.386
19.774
19.99
299.992
59.994
66.082
40.0
259.991
140.025
60.006
659.976
1000.01
1200.007
120.001
19.982
0.0
0.0
0.0
39.984
159.946
0.0
0.0
79.99
19.995
0.0
120.012
19.993
259.989
99.983
160.02
19.967
0.0
40.012
0.0
399.99
79.986
60.01
319.986
19.99
0.0
19.994
0.0
0.0
0.0
0.0
100.005
19.993
0.0
19.999
39.969
0.0
41.073
220.015
60.019
80.857
19.986
40.018
0.0
99.983
279.957
200.023
0.0
20.

180.006
18.012
279.92
1589.455
33.275
1753.517
80.036
0.0
60.003
39.98
59.998
40.032
119.982
120.004
0.0
0.0
30.059
20.022
20.035
19.989
0.0
0.0
40.002
59.982
0.0
0.0
0.0
19.977
0.0
40.039
0.0
0.0
39.967
99.972
40.03
1300.666
90.816
2540.248
3994.163
152.924
0.0
0.0
20.023
0.0
0.0
0.0
0.0
30.792
119.998
19.953
280.012
40.008
580.699
39.997
439.981
0.0
0.0
0.0
65.017
0.0
13.277
0.0
120.009
235.718
520.009
0.0
12.161
11.315
149.114
13.308
26.982
0.0
2.011
20.274
0.0
13.308
0.0
0.0
19.13
2.042
0.0
2.949
10.146
0.0
0.0
59.631
0.0
582.598
0.0
0.0
0.0
110.694
567.508
22.971
7.5
21.573
0.0
0.0
0.0
0.0
0.0
139.98
499.818
0.0
0.0
38.676
31.561
2.148
62.13
2.184
31.654
2.749
18.729
0.0
46.571
4.704
0.0
2.023
0.0
0.0
2.608
0.0
97.017
0.0
2.001
9.383
11.358
0.0
5.487
0.0
29.13
0.0
6.966
18.249
39.115
3.261
0.0
20.28
37.431
82.277
36.067
22.08
32.193
0.0
0.0
4.299
199.686
0.0
0.0
21.286
17.93
55.519
0.0
0.0
2.013
0.0
0.0
621.712
0.0
2.008
14.265
0.0
0.0
0.0
0.0
0.0
2.01
0.0
0.0
2.025
0.0
0.0
0.0
24

0.0
18.656
99.948
0.0
6.112
16.961
40.017
20.022
0.0
100.006
313.069
7.141
8.144
19.966
39.971
14.855
0.0
24.978
180.004
34.97
14.979
44.816
271.011
78.982
2140.109
42.115
39.006
10.887
4.063
19.988
27.957
8.188
179.97
39.995
39.963
577.933
19.997
79.977
74.493
9.989
0.0
0.0
37.998
6261.327
0.0
239.981
0.0
18.999
45.96
141.295
82.894
19.838
19.97
160.434
79.994
1.015
0.0
0.0
19.983
60.011
0.0
20.015
0.0
139.974
19.952
0.0
19.953
39.984
59.6
19.984
19.993
0.0
80.015
5.11
39.99
6.152
0.0
82.917
0.0
0.0
0.0
0.0
60.787
20.497
99.994
0.0
0.0
0.0
20.015
39.995
0.0
0.0
140.007
58.965
40.706
399.001
59.996
589.694
460.618
706.49
380.454
220.008
199.515
319.977
100.083
140.168
680.349
399.999
699.781
0.0
95.921
19.996
8.149
47.921
129.13
15.981
59.986
59.963
78.997
0.0
14.947
19.988
19.936
20.025
0.0
40.009
0.0
0.0
0.0
3.044
0.0
119.974
0.0
40.053
0.0
0.0
9.242
0.0
0.0
45.94
0.0
0.0
40.006
159.975
28.945
0.0
40.003
0.0
82.027
80.104
29.969
0.0
0.0
24.95
73.981
0.0
40.034
181.062
0.0
20.008
6.06

0.0
0.0
0.0
7.088
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
34.935
13.92
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.045
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.047
40.003
0.0
0.0
0.0
19.994
0.0
0.0
0.0
0.0
0.0
0.0
5.133
0.0
0.0
19.998
0.0
0.0
0.0
0.0
0.0
0.0
0.0
19.993
0.0
0.0
0.0
0.0
20.022
0.0
0.0
0.0
19.989
4.103
5.092
0.0
0.0
0.0
0.0
0.0
19.97
0.0
0.0
0.0
0.0
9.128
0.0
0.0
5.045
0.0
0.0
19.973
0.0
0.0
0.0
9.265
0.0
20.016
0.0
0.0
0.0
0.0
19.035
0.0
0.0
0.0
0.0
20.011
80.02
0.0
20.007
19.999
0.0
0.0
40.004
59.999
19.995
0.0
39.971
0.0
20.013
0.0
0.0
20.03
19.983
20.021
0.0
19.975
39.989
19.978
0.0
0.0
20.01
79.963
0.0
0.0
19.972
79.954
19.976
26.058
0.0
9.253
39.999
0.0
19.997
52.127
39.551
73.959
100.031
0.0
9.16
12.923
1.003
56.962
71.951
91.154
80.007
15.925
0.0
0.0
40.007
0.0
60.021
79.998
0.0
80.015
0.0
49.087
0.0
19.965
29.791
20.014
0.0
19.997
0.0
0.0
0.0
0.0
19.996
0.0
19.977
8.098
13.965
9.105
99.988
20.02
19.99
0.0
39.987
39.987
19.991
40.001
99.988
20.014
0.0
19.998
0.0
20.015
39.98
0.0
0.0
0.0
1

56.341
60.17
93.232
88.326
12.108
0.0
9.882
47.794
11.189
0.0
127.605
9.972
29.928
13.047
0.0
21.263
10.267
15.469
0.0
0.0
0.0
0.0
0.0
2.541
90.234
69.9
126.711
40.045
0.0
0.0
73.364
16.311
235.467
70.142
332.61
243.967
9.182
0.0
35.055
12.931
66.99
0.0
58.611
189.507
107.828
71.38
74.479
10.549
43.77
4.594
190.529
15.688
151.371
173.943
153.822
44.794
67.249
296.618
46.254
49.819
53.451
33.261
32.149
54.787
32.221
243.126
187.393
63.369
833.311
161.304
33.879
233.208
392.318
193.578
109.186
51.134
72.743
134.415
212.703
314.81
290.85
56.081
432.762
554.046
336.515
290.93
174.773
216.594
69.115
234.849
54.442
221.221
592.963
309.656
491.694
266.848
246.462
58.839
0.0
358.411
33.276
127.943
133.268
169.708
87.925
0.0
79.77
132.845
111.014
154.776
292.278
84.087
0.0
0.0
90.088
128.588
273.658
14.419
215.174
155.878
3.096
133.637
447.426
9.891
255.428
1013.437
371.805
1054.769
355.83
673.345
247.952
94.528
33.254
0.0
0.0
69.095
394.325
33.628
124.769
29.182
0.0
60.896
8.756
19.014
45.902
